In [1]:
import sys
sys.path.insert(1, '../..')

import torch
import torch.nn as nn
import random
import pandas as pd
import numpy as np

from library.evaluation import ConfusionMatrix

random.seed(33)

dataset_name = "Twitter15_CrossVal"
unique_name = "T5"

In [2]:
vectors = np.loadtxt("../../data/processed/vectors/Twitter15_T5_vectors.txt", delimiter=",")
first = vectors[0]
vectors.shape

(1490, 768)

In [3]:
data = pd.read_csv("../../data/processed/twitter15_dataset_with_tvt.csv", lineterminator="\n")
data.head()

,tweet_id,tweet_text,label,tvt,cv_fold,tt
0,731166399389962242,🔥ca kkk grand wizard 🔥 endorses @hillaryclinto...,unverified,training,1,training
1,714598641827246081,an open letter to trump voters from his top st...,unverified,training,1,test
2,691809004356501505,america is a nation of second chances —@potus ...,non-rumor,training,2,training
3,693204708933160960,"brandon marshall visits and offers advice, sup...",non-rumor,training,1,training
4,551099691702956032,rip elly may clampett: so sad to learn #beverl...,true,training,3,training


In [4]:
labels_str = data['label'].unique().tolist()
labels_str

['unverified', 'non-rumor', 'true', 'false']

In [5]:
labels = []
for i, d in data.iterrows():
    lab = labels_str.index(d['label'])
    labels.append(lab)
labels[:10]

[0, 0, 1, 1, 2, 1, 0, 2, 0, 3]

In [6]:
test_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])
test_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] == 0])

In [7]:
print(test_vectors.shape)
print(test_labels.shape)

(151, 768)
(151,)


In [8]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes):
        super(BasicBlock, self).__init__()
        self.lin1 = nn.Linear(in_planes, planes)
        self.bn1 = nn.BatchNorm1d(planes)
        self.lin2 = nn.Linear(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Linear(in_planes, planes),
                nn.BatchNorm1d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = self.bn2(self.lin2(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes):
        super(Bottleneck, self).__init__()
        self.lin1 = nn.Linear(in_planes, planes)
        self.bn1 = nn.BatchNorm1d(planes)
        self.lin2 = nn.Linear(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.lin3 = nn.Linear(planes, planes)
        self.bn3 = nn.BatchNorm1d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Linear(in_planes, planes),
                nn.BatchNorm1d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = F.leaky_relu(self.bn2(self.lin2(out)), 0.1)
        out = self.bn3(self.lin3(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class ResNet(nn.Module):
    def __init__(self, block, num_blocks, n_input=768, num_classes=1):
        super(ResNet, self).__init__()
        self.in_planes = 512

        self.lin1 = nn.Linear(n_input, self.in_planes)
        self.bn1 = nn.BatchNorm1d(self.in_planes)
        self.layer1 = self._make_layer(block, 512, num_blocks[0])
        self.layer2 = self._make_layer(block, 256, num_blocks[1])
        self.layer3 = self._make_layer(block, 128, num_blocks[2])
        self.layer4 = self._make_layer(block, 64, num_blocks[3])
        self.linear = nn.Linear(64, num_classes)

    def _make_layer(self, block, planes, num_blocks):
        strides = [1] * num_blocks
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes))
            self.in_planes = planes
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def ResNet10(n_input=768, n_output=1, block=BasicBlock):
    return ResNet(block, [1, 1, 1, 1], n_input, n_output)

    
def ResNet18(n_input=768, n_output=1, block=BasicBlock):
    return ResNet(block, [2, 2, 2, 2], n_input, n_output)


def ResNet34(n_input=768, n_output=1, block=BasicBlock):
    return ResNet(block, [3, 4, 6, 3], n_input, n_output)


def ResNet50(n_input=768, n_output=1, block=Bottleneck):
    return ResNet(block, [3, 4, 6, 3], n_input, n_output)


def ResNet101(n_input=768, n_output=1, block=Bottleneck):
    return ResNet(block, [3, 4, 23, 3], n_input, n_output)


def ResNet152(n_input=768, n_output=1, block=Bottleneck):
    return ResNet(block, [3, 8, 36, 3], n_input, n_output)

In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class CNNBasicBlock(nn.Module):
    expansion = 1

    def __init__(self, in_planes, planes, stride=1):
        super(CNNBasicBlock, self).__init__()
        self.conv1 = nn.Conv2d(
            in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, self.expansion*planes,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)), 0.1)
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class CNNBottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_planes, planes):
        super(CNNBottleneck, self).__init__()
        self.lin1 = nn.Linear(in_planes, planes)
        self.bn1 = nn.BatchNorm2d(planes)
        self.lin2 = nn.Linear(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.lin3 = nn.Linear(planes, planes)
        self.bn3 = nn.BatchNorm2d(planes)

        self.shortcut = nn.Sequential()
        if in_planes != planes:
            self.shortcut = nn.Sequential(
                nn.Linear(in_planes, planes),
                nn.BatchNorm1d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.lin1(x)), 0.1)
        out = F.leaky_relu(self.bn2(self.lin2(out)), 0.1)
        out = self.bn3(self.lin3(out))
        out += self.shortcut(x)
        out = F.leaky_relu(out, 0.1)
        return out


class CNNResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=1):
        super(CNNResNet, self).__init__()
        self.in_planes = 24

        self.conv1 = nn.Conv2d(1, 24, kernel_size=3,
                               stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(24)
        self.layer1 = self._make_layer(block, 24, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 32, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 48, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 64, num_blocks[3], stride=2)
        self.linear = nn.Linear(64 * 24 * 32, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1]*(num_blocks-1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes))
            self.in_planes = planes * block.expansion
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)), 0.1)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


def CNNResNet10(n_output: int):
    return CNNResNet(CNNBasicBlock, [1, 1, 1, 1], n_output)

    
def CNNResNet18(n_output: int):
    return CNNResNet(CNNBasicBlock, [2, 2, 2, 2], n_output)


def CNNResNet34(n_output: int):
    return CNNResNet(CNNBasicBlock, [3, 4, 6, 3], n_output)


def CNNResNet50(n_output: int):
    return CNNResNet(CNNBottleneck, [3, 4, 6, 3], n_output)


def CNNResNet101(n_output: int):
    return CNNResNet(CNNBottleneck, [3, 4, 23, 3], n_output)


def CNNResNet152(n_output: int):
    return CNNResNet(CNNBottleneck, [3, 8, 36, 3], n_output)

In [10]:
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torch.optim as optim
import matplotlib.pyplot as plt
import time
import os
from typing import Callable


class ResNetClassifier(nn.Module):
    def __init__(self,
        model,
        n_input: int,
        n_output: int = 1,
        criterion: Callable = nn.BCELoss,
        n_features: int = 4,
        lr: float = 0.0002,
        beta1: float = 0.5,
        device: str = None,
        model_type: str = "mlp"
    ):
        super(ResNetClassifier, self).__init__()
        self.model = model
        self.model_type = model_type
        self.criterion = criterion()
        self.optimizer = optim.Adam(self.model.parameters(), lr=lr, betas=(beta1, 0.999))
        
        if not device or device not in ['cpu', 'cuda']:
            self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        else:
            self.device = device

        self.model = self.model.to(self.device)
        if self.device == 'cuda':
            self.model = torch.nn.DataParallel(self.model)
            cudnn.benchmark = True

    def forward(self, input):
        x = self.model(input)
        return x
    
    def load_pretrained(self, filepath: str, key: str = "net", is_parallel: bool = False):
        checkpoint = torch.load(filepath)
        if is_parallel:
            self.model = torch.nn.DataParallel(self.model)
        self.model.load_state_dict(checkpoint[key], strict=False)
    
    def save_model(self, saves: str):
        print(f"Saving model...")
        state = {
            'net': self.model.state_dict()
        }
        if not os.path.isdir('models'):
            os.mkdir('models')
        torch.save(state, f"../../data/models/{saves}.pth")
    
    def train_eval(self,
        train_x, train_y,
        test_x, test_y,
        n_iter: int = 100,
        batch_size: int = 128,
        saves: str = None
    ):
        trainset = torch.utils.data.TensorDataset(train_x, train_y) # create your datset
        trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size) # create your dataloader

        testset = torch.utils.data.TensorDataset(test_x, test_y) # create your datset
        testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size) # create your dataloader

        train_accs = []
        train_losses = []
        test_accs = []
        test_losses = []

        print(f"Using {self.device}")
        best_acc = 0
        current_loss = 1000
        best_test_acc = 0
        epoch = 0
        start_time = time.time()
        results = {}
        while True:
            epoch += 1
            self.model.train()
            train_loss = 0
            correct = 0
            total = 0
            for batch_idx, (inputs, targets) in enumerate(trainloader):
                self.model.zero_grad()
                inputs, targets = inputs.to(self.device), targets.to(self.device)

                if self.model_type == "cnn":
                    outputs = self.model(inputs.unsqueeze(1))
                elif self.model_type == "mlp":
                    outputs = self.model(inputs)

                loss = self.criterion(outputs, targets.long())
                loss.backward()
                self.optimizer.step()

                train_loss += loss.item()
                total += targets.size(0)
#                 for i, output in enumerate(outputs.tolist()):
#                     if targets[i,0].tolist() == round(output[0]):
#                         correct += 1

#             train_acc = round(100*correct/total, 4)
#             train_accs.append(train_acc)
            train_losses.append(train_loss)

            self.model.eval()
            test_loss = 0
            test_acc = 0
            with torch.no_grad():
                inputs, targets = test_x.to(self.device), test_y.to(self.device)
                
                if self.model_type == 'mlp':
                    outputs = self.model(inputs)
                else:
#                     outputs = self.model(inputs.reshape(inputs.shape[0], 1, 24, 32))
                    outputs = self.model(inputs.unsqueeze(1))

                loss = self.criterion(outputs, targets.long())
                test_loss += loss.item()
                
                if self.model_type == 'mlp':
                    preds = self.predict(test_x)
                else:
                    preds = self.predict(test_x.reshape(test_x.shape[0], 1, 24, 32))
                conf_mat = ConfusionMatrix(
                    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_y]),
                    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds.cpu().numpy()]),
                    binary=False
                )
                conf_mat.evaluate(logs=False)
                test_acc = conf_mat.accuracy

            test_losses.append(test_loss)
            
#             if (epoch) % round(n_iter/20) == 0:
#                 print(f"-- Epoch {epoch}, Train Loss : {train_loss}, Test Loss : {test_loss}")

            # Save checkpoint.
#             if saves and test_loss < best_loss:
#                 print(f"Saving after new best loss : {test_loss}")
#                 best_loss = test_loss
            if saves and test_acc > best_test_acc:
#                 print(f"Saving after new best accuracy : {test_acc}")
                best_test_acc = test_acc

                state = {
                    'net': self.model.state_dict(),
                }
                if not os.path.isdir('models'):
                    os.mkdir('models')
                torch.save(state, f"../../data/models/{saves}.pth")
            
            if epoch >= n_iter:
                break

#         # visualizing accuracy over epoch
#         fig, ax2 = plt.subplots(1)
#         plt.subplots_adjust(top = 0.99, bottom=0.01, hspace=1.5, wspace=0.4)

#         ax2.plot([i for i in range(len(train_losses))], train_losses, c='b', marker="o", label='Train Loss')
#         ax2.plot([i for i in range(len(test_losses))], test_losses, c='r', marker="o", label='Test Loss')
#         ax2.set_ylabel('Loss')
#         ax2.set_xlabel('Epoch')
#         ax2.set_xlim(0, len(train_losses))
#         ax2.set_ylim(min([min(train_losses), min(test_losses)])*0.1, max([max(train_losses), max(test_losses)]))
#         ax2.title.set_text(f"Loss over time (epoch)")
#         ax2.legend(loc='lower right')

#         plt.show()
    
    def predict(self, input_x):
        self.model.eval()
        with torch.no_grad():
            preds = self.model(torch.Tensor(input_x))
            preds = torch.log_softmax(preds, dim = 1)
            _, preds = torch.max(preds, dim = 1)
            return preds

In [11]:
print("3-Fold Cross Validation with ResNet10 CNN")

folds = [1, 2, 3]
for fold in folds:
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold}"
    print(f"Multiclass Classification {model_name}")
    model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")
    model.train_eval(torch.Tensor(train_vectors.reshape(train_vectors.shape[0], 24, 32)),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors.reshape(val_vectors.shape[0], 24, 32)),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=1024)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors.reshape(val_vectors.shape[0], 1, 24, 32))

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with ResNet10 CNN
Fold-1 Cross Validation
Multiclass Classification Twitter15_CrossVal_ResNet10_CNN_T5_fold-1
Using cuda

-------- Fold-1 Results --------
474 vs 474
Multi Class Evaluation

Class unverified Evaluation
- Precision : 67.257 %
- Recall : 64.407 %
- F1 : 0.65801

Class non-rumor Evaluation
- Precision : 64.0 %
- Recall : 72.727 %
- F1 : 0.68085

Class true Evaluation
- Precision : 79.825 %
- Recall : 73.387 %
- F1 : 0.76471

Class false Evaluation
- Precision : 72.951 %
- Recall : 72.951 %
- F1 : 0.72951

Combined Evaluation
- Accuracy : 70.886 %
- Precision : 71.008 %
- Recall : 70.868 %
- F1 : 0.70938

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet10_CNN_T5_fold-1, 70.886, 71.008, 70.868, 0.70938, 67.257, 64.407, 0.65801, 64.0, 72.727, 0.68085, 79.825, 73.387, 0.76471, 72.951, 72.951, 0.72951, 
-------- Fold-1 End --------

Fold-2 Cross Validation
Multiclass Classification Twitt

In [12]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 54.054 %
- Recall : 55.556 %
- F1 : 0.54795

Class non-rumor Evaluation
- Precision : 60.0 %
- Recall : 78.947 %
- F1 : 0.68182

Class true Evaluation
- Precision : 81.818 %
- Recall : 69.231 %
- F1 : 0.75

Class false Evaluation
- Precision : 67.742 %
- Recall : 55.263 %
- F1 : 0.6087

Combined Evaluation
- Accuracy : 64.901 %
- Precision : 65.904 %
- Recall : 64.749 %
- F1 : 0.65321

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet10_CNN_T5_fold-1, 64.901, 65.904, 64.749, 0.65321, 54.054, 55.556, 0.54795, 60.0, 78.947, 0.68182, 81.818, 69.231, 0.75, 67.742, 55.263, 0.6087, 
-------- Testing End --------



In [13]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 54.839 %
- Recall : 47.222 %
- F1 : 0.50746

Class non-rumor Evaluation
- Precision : 59.615 %
- Recall : 81.579 %
- F1 : 0.68889

Class true Evaluation
- Precision : 67.568 %
- Recall : 64.103 %
- F1 : 0.65789

Class false Evaluation
- Precision : 67.742 %
- Recall : 55.263 %
- F1 : 0.6087

Combined Evaluation
- Accuracy : 62.252 %
- Precision : 62.441 %
- Recall : 62.042 %
- F1 : 0.62241

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet10_CNN_T5_fold-2, 62.252, 62.441, 62.042, 0.62241, 54.839, 47.222, 0.50746, 59.615, 81.579, 0.68889, 67.568, 64.103, 0.65789, 67.742, 55.263, 0.6087, 
-------- Testing End --------



In [14]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet10_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet10(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 64.286 %
- Recall : 50.0 %
- F1 : 0.5625

Class non-rumor Evaluation
- Precision : 62.222 %
- Recall : 73.684 %
- F1 : 0.6747

Class true Evaluation
- Precision : 69.767 %
- Recall : 76.923 %
- F1 : 0.73171

Class false Evaluation
- Precision : 65.714 %
- Recall : 60.526 %
- F1 : 0.63014

Combined Evaluation
- Accuracy : 65.563 %
- Precision : 65.497 %
- Recall : 65.283 %
- F1 : 0.6539

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet10_CNN_T5_fold-3, 65.563, 65.497, 65.283, 0.6539, 64.286, 50.0, 0.5625, 62.222, 73.684, 0.6747, 69.767, 76.923, 0.73171, 65.714, 60.526, 0.63014, 
-------- Testing End --------



In [15]:
print("3-Fold Cross Validation with ResNet18 CNN")

folds = [1, 2, 3]
for fold in folds:
    val_folds = [fold]
    train_folds = folds.copy()
    train_folds.remove(fold)

    train_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_vectors = np.array([vectors[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    train_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in train_folds])
    val_labels = np.array([labels[i] for i, p2 in data.iterrows() if p2['cv_fold'] in val_folds])

    print(f"Fold-{fold} Cross Validation")
    model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold}"
    print(f"Multiclass Classification {model_name}")
    model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")
    model.train_eval(torch.Tensor(train_vectors.reshape(train_vectors.shape[0], 24, 32)),
                    torch.Tensor(train_labels),
                    torch.Tensor(val_vectors.reshape(val_vectors.shape[0], 24, 32)),
                    torch.Tensor(val_labels),
                    saves=model_name,
                    n_iter=1000,
                    batch_size=1024)

    model.load_pretrained(f"../../data/models/{model_name}.pth")

    print(f"\n-------- Fold-{fold} Results --------")
    preds = model.predict(val_vectors.reshape(val_vectors.shape[0], 1, 24, 32))

    preds = preds.cpu().numpy()

    conf_mat = ConfusionMatrix(
        labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in val_labels]),
        predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
        binary=False,
        model_name=model_name
    )
    conf_mat.evaluate(classes=labels_str)
    print(f"-------- Fold-{fold} End --------\n")

3-Fold Cross Validation with ResNet18 CNN
Fold-1 Cross Validation
Multiclass Classification Twitter15_CrossVal_ResNet18_CNN_T5_fold-1
Using cuda

-------- Fold-1 Results --------
474 vs 474
Multi Class Evaluation

Class unverified Evaluation
- Precision : 60.833 %
- Recall : 61.864 %
- F1 : 0.61345

Class non-rumor Evaluation
- Precision : 59.2 %
- Recall : 67.273 %
- F1 : 0.62979

Class true Evaluation
- Precision : 74.312 %
- Recall : 65.323 %
- F1 : 0.69528

Class false Evaluation
- Precision : 65.833 %
- Recall : 64.754 %
- F1 : 0.65289

Combined Evaluation
- Accuracy : 64.768 %
- Precision : 65.045 %
- Recall : 64.803 %
- F1 : 0.64924

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet18_CNN_T5_fold-1, 64.768, 65.045, 64.803, 0.64924, 60.833, 61.864, 0.61345, 59.2, 67.273, 0.62979, 74.312, 65.323, 0.69528, 65.833, 64.754, 0.65289, 
-------- Fold-1 End --------

Fold-2 Cross Validation
Multiclass Classification Twitt

In [16]:
fold_n = 1
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-1

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 63.636 %
- Recall : 58.333 %
- F1 : 0.6087

Class non-rumor Evaluation
- Precision : 62.264 %
- Recall : 86.842 %
- F1 : 0.72527

Class true Evaluation
- Precision : 72.222 %
- Recall : 66.667 %
- F1 : 0.69333

Class false Evaluation
- Precision : 72.414 %
- Recall : 55.263 %
- F1 : 0.62687

Combined Evaluation
- Accuracy : 66.887 %
- Precision : 67.634 %
- Recall : 66.776 %
- F1 : 0.67202

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet18_CNN_T5_fold-1, 66.887, 67.634, 66.776, 0.67202, 63.636, 58.333, 0.6087, 62.264, 86.842, 0.72527, 72.222, 66.667, 0.69333, 72.414, 55.263, 0.62687, 
-------- Testing End --------



In [17]:
fold_n = 2
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-2

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 57.143 %
- Recall : 55.556 %
- F1 : 0.56338

Class non-rumor Evaluation
- Precision : 62.745 %
- Recall : 84.211 %
- F1 : 0.7191

Class true Evaluation
- Precision : 79.412 %
- Recall : 69.231 %
- F1 : 0.73973

Class false Evaluation
- Precision : 70.968 %
- Recall : 57.895 %
- F1 : 0.63768

Combined Evaluation
- Accuracy : 66.887 %
- Precision : 67.567 %
- Recall : 66.723 %
- F1 : 0.67142

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet18_CNN_T5_fold-2, 66.887, 67.567, 66.723, 0.67142, 57.143, 55.556, 0.56338, 62.745, 84.211, 0.7191, 79.412, 69.231, 0.73973, 70.968, 57.895, 0.63768, 
-------- Testing End --------



In [18]:
fold_n = 3
print(f"Testing Cross Validation Fold-{fold_n}")

model_name = f"{dataset_name}_ResNet18_CNN_{unique_name}_fold-{fold_n}"
model = ResNetClassifier(CNNResNet18(n_output=4), train_vectors.shape[1], n_output=4, criterion=nn.CrossEntropyLoss, n_features=16, model_type="cnn") #, device="cpu")

model.load_pretrained(f"../../data/models/{model_name}.pth")

print(f"\n-------- Testing Results --------")
preds = model.predict(test_vectors.reshape(test_vectors.shape[0], 1, 24, 32))

preds = preds.cpu().numpy()

conf_mat = ConfusionMatrix(
    labels=np.array([[1 if j == v else 0 for j in range(len(labels_str))] for v in test_labels]),
    predictions=np.array([[1 if j == p else 0 for j in range(len(labels_str))] for p in preds]),
    binary=False,
    model_name=model_name
)
conf_mat.evaluate(classes=labels_str)
print(f"-------- Testing End --------\n")

Testing Cross Validation Fold-3

-------- Testing Results --------
151 vs 151
Multi Class Evaluation

Class unverified Evaluation
- Precision : 59.375 %
- Recall : 52.778 %
- F1 : 0.55882

Class non-rumor Evaluation
- Precision : 60.0 %
- Recall : 78.947 %
- F1 : 0.68182

Class true Evaluation
- Precision : 73.684 %
- Recall : 71.795 %
- F1 : 0.72727

Class false Evaluation
- Precision : 67.742 %
- Recall : 55.263 %
- F1 : 0.6087

Combined Evaluation
- Accuracy : 64.901 %
- Precision : 65.2 %
- Recall : 64.696 %
- F1 : 0.64947

- Average Confidence : 100.0 %
Model, Combined,,,,unverified,,,non-rumor,,,true,,,false,,,
Twitter15_CrossVal_ResNet18_CNN_T5_fold-3, 64.901, 65.2, 64.696, 0.64947, 59.375, 52.778, 0.55882, 60.0, 78.947, 0.68182, 73.684, 71.795, 0.72727, 67.742, 55.263, 0.6087, 
-------- Testing End --------

